# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, WT, Male, VISl, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISp and VISl sessions. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('wt/wt') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISl' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"
739448407,2019-10-03T00:00:00Z,716813543,brain_observatory_1.1,112.0,M,wt/wt,625,2221,6,"[grey, VISam, nan, VIS, VISp, VISl, VISrl]"
742951821,2019-10-03T00:00:00Z,723627604,brain_observatory_1.1,120.0,M,wt/wt,893,2219,6,"[VISal, nan, grey, VISl, VISrl, VISp, VISpm, VIS]"
743475441,2019-10-03T00:00:00Z,722882755,brain_observatory_1.1,121.0,M,wt/wt,553,2225,6,"[LP, LGd, HPF, DG, CA3, CA1, VISrl, nan, PP, P..."


In [7]:
session_id = filtered_sessions.index.values[4]

session = cache.get_session_data(session_id)

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

Total number of units: 553


,waveform_spread,isolation_distance,waveform_repolarization_slope,cluster_id,nn_miss_rate,waveform_PT_ratio,silhouette_score,waveform_recovery_slope,d_prime,max_drift,amplitude_cutoff,waveform_velocity_below,waveform_duration,waveform_velocity_above,firing_rate,L_ratio,waveform_halfwidth,nn_hit_rate,isi_violations,presence_ratio,cumulative_drift,snr,local_index_unit,waveform_amplitude,peak_channel_id,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
950991672,40.0,114.048152,0.800524,0,0.001726,0.395276,NaN,-0.146184,6.247022,38.84,0.001726,NaN,0.206030,-0.137353,33.236768,4.407722e-04,0.109883,0.994667,0.011538,0.99,278.89,3.986705,0,88.484955,849755600,NaN,500.0,1.167677,2.204513,2.313982,1.384127,1.203767,0.344330,0.021697,0.030969,0.022221,14.0,2.295002,0.039,0.32,15.0,-0.010309,NaN,-0.034772,NaN,0.045113,90.0,30.0,0.886904,NaN,0.766098,NaN,8.055643e-01,42.000,4998,0.25,32.539828,34.318295,31.309623,37.250204,31.506325,NaN,0.1985,False,False,False,False,False,0.033729,False,0.003163,0.000391,0.004013,0.013227,0.004769,20,11,769325777,1,215.0,APN,7918.0,3358.0,6665.0,probeA,See electrode locations,29999.967105,1249.998629,True
950991690,40.0,88.061548,0.364325,2,0.001427,0.929605,0.200260,-0.051021,4.639649,48.62,0.001257,0.000000,0.343384,-0.343384,19.613082,7.833651e-04,0.151089,0.989333,0.009169,0.99,488.30,2.780966,2,134.609670,849755606,NaN,400.0,31.218917,4.592821,6.426334,6.729825,5.283481,0.177062,0.013248,0.117594,0.012677,67.5,0.104939,0.365,0.04,2.0,NaN,NaN,-0.407441,-0.638889,-0.683504,315.0,150.0,NaN,NaN,0.014761,0.048571,6.587679e-07,10.000,4991,0.25,19.528980,15.347312,20.521331,12.039540,23.279204,NaN,0.1115,False,False,False,False,False,0.026153,False,0.063977,0.034223,0.017243,0.057325,0.020223,60,43,769325777,4,215.0,APN,7911.0,3331.0,6673.0,probeA,See electrode locations,29999.967105,1249.998629,True
950991717,50.0,151.647329,1.110627,5,0.000459,0.499588,0.269976,-0.311191,5.968511,33.70,0.000575,-2.060302,0.288442,0.618090,27.574704,5.206772e-05,0.151089,1.000000,0.302836,0.99,121.86,3.719315,5,280.221630,849755610,NaN,600.0,8.078604,8.475762,6.103969,5.684496,6.767902,0.461560,0.060492,0.164803,0.042311,80.0,2.875865,0.000,0.02,15.0,NaN,NaN,-0.684314,0.241379,-0.709302,270.0,90.0,NaN,NaN,0.028067,0.380694,6.173374e-06,31.667,4944,0.25,24.058843,19.876901,30.111968,21.059326,30.799583,NaN,0.1775,False,False,False,False,False,0.097576,False,0.075260,0.018458,0.016970,0.113218,0.026075,80,59,769325777,6,215.0,APN,7906.0,3314.0,6678.0,probeA,See electrode locations,29999.967105,1249.998629,True
950991752,60.0,94.931602,0.960801,8,0.004174,0.453972,0.212655,-0.199891,4.822588,18.49,0.000638,0.000000,0.192295,-0.274707,5.577156,1.630797e-03,0.123618,0.929563,0.453583,0.96,134.55,3.789455,8,227.825130,849755616,NaN,300.0,24.643011,5.039497,7.361875,9.302954,8.609180,0.741804,0.101335,0.067580,0.039399,90.0,3.061182,0.220,0.32,2.0,-0.915033,NaN,-1.000000,NaN,-0.801653,315.0,90.0,0.008189,NaN,0.0001

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISl') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 14


,waveform_spread,isolation_distance,waveform_repolarization_slope,cluster_id,nn_miss_rate,waveform_PT_ratio,silhouette_score,waveform_recovery_slope,d_prime,max_drift,amplitude_cutoff,waveform_velocity_below,waveform_duration,waveform_velocity_above,firing_rate,L_ratio,waveform_halfwidth,nn_hit_rate,isi_violations,presence_ratio,cumulative_drift,snr,local_index_unit,waveform_amplitude,peak_channel_id,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951006697,50.0,60.960179,0.282340,226,0.001066,0.398827,0.117836,-0.033608,4.519006,22.37,0.015079,-0.686767,0.659296,0.343384,1.469410,0.004281,0.178559,0.964169,0.055690,0.99,237.43,2.907866,220,88.174710,849752312,NaN,400.0,6.118519,1.216970,2.032222,7.583740,2.706019,0.309793,0.159861,0.769494,0.517113,15.0,0.349704,0.00,0.08,2.0,NaN,NaN,0.728682,0.920228,0.709756,45.0,60.0,NaN,NaN,0.009891,0.117274,0.010148,5.0,4972,0.75,2.183124,0.346380,0.900760,1.254393,2.098246,NaN,0.1115,False,False,False,False,False,0.686407,False,0.687101,0.648000,0.354605,0.570729,0.493196,2220,11,769326325,221,409.0,VISl,8495.0,1919.0,9100.0,probeD,See electrode locations,29999.92056,1249.99669,True
951006813,60.0,65.153110,0.120040,235,0.008029,0.381648,-0.001936,-0.006153,3.689122,32.50,0.015996,-0.686767,0.700503,-0.618090,8.469882,0.015484,0.302178,0.920667,0.056988,0.99,321.02,1.188582,229,42.677115,849752320,NaN,100.0,4.904274,2.933333,1.099717,2.574359,1.846690,0.653896,0.071084,0.240067,0.027091,10.0,0.461453,0.00,0.32,15.0,NaN,NaN,-0.029412,NaN,0.424419,225.0,60.0,NaN,NaN,0.810334,NaN,0.013861,0.0,4980,0.00,6.906255,6.474647,13.937927,1.719308,12.309708,NaN,0.1155,False,False,False,False,False,-0.035712,False,0.262192,0.104305,0.064832,0.279378,0.211066,2260,11,769326325,225,409.0,VISl,8512.0,1888.0,9121.0,probeD,See electrode locations,29999.92056,1249.99669,True
951006776,50.0,65.233634,0.636410,232,0.000169,0.378288,0.305598,-0.088372,5.587359,10.44,0.000019,-0.686767,0.604355,2.060302,1.461390,0.001148,0.137353,0.936170,0.056303,0.99,66.17,4.464423,226,194.928240,849752320,NaN,200.0,2.200433,0.980513,0.905644,1.233333,1.172174,0.427392,0.030499,0.865116,0.319955,50.0,0.630885,0.09,0.02,2.0,NaN,NaN,0.247795,0.250000,0.346154,225.0,60.0,NaN,NaN,0.324658,0.838779,0.532058,5.0,5023,0.00,1.385699,0.532893,3.419662,0.135966,3.383838,NaN,0.0885,False,False,False,False,False,0.499424,False,0.715947,0.165138,0.372816,0.683237,0.445075,2260,11,769326325,225,409.0,VISl,8512.0,1888.0,9121.0,probeD,See electrode locations,29999.92056,1249.99669,True
951006841,50.0,81.734936,0.256481,237,0.011625,0.567376,0.091022,-0.045311,3.222817,52.12,0.071809,-0.343384,0.370854,0.686767,13.921596,0.004121,0.178559,0.968000,0.051908,0.99,227.69,2.009417,231,64.986870,849752322,NaN,500.0,10.986091,1.367026,0.896322,3.139394,2.418548,0.422528,0.048007,0.121433,0.073730,14.0,0.719796,0.00,0.04,1.0,0.545749,NaN,0.270531,0.372781,0.340278,45.0,60.0,0.001006,NaN,0.022089,0.613351,0.489974,8.0,4939,0.75,11.617

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
951006697     2.098246
951006813    12.309708
951006776     3.383838
951006841    15.006121
951006827    23.725497
951006880    18.675719
951008476     0.752038
951006965     0.165861
951006949    17.731842
951007158     9.908383
951007370    11.075407
951007438     8.463591
951007395     7.980661
951007489     8.157180
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

9.959577957157904


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
951006697     2.183124
951006813     6.906255
951006776     1.385699
951006841    11.617207
951006827    23.538875
951006880    15.760536
951008476     0.418834
951006965     0.133963
951006949    14.181570
951007158     8.644600
951007370     9.074553
951007438     5.332584
951007395     4.454145
951007489     7.438931
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

7.933633894569167


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])